# Explore here

In [5]:
import pandas as pd

import re

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords');
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet');

from tqdm import tqdm
tqdm.pandas();

from wordcloud import WordCloud

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.svm import SVC

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.metrics import classification_report, accuracy_score

from scipy.stats import uniform

[nltk_data] Downloading package stopwords to /home/vscode/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/vscode/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:

df = pd.read_csv ('https://raw.githubusercontent.com/4GeeksAcademy/NLP-project-tutorial/main/url_spam.csv', sep = ",");

pd.set_option('display.max_columns', None);

df.head()

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,True
1,https://www.hvper.com/,True
2,https://briefingday.com/m/v4n3i4f3,True
3,https://briefingday.com/n/20200618/m#commentform,False
4,https://briefingday.com/fan,True


In [ ]:
# Numero de URLS
df.shape[0]

2999

In [ ]:
# Tipod de variables
print(df.dtypes)

url        object
is_spam      bool
dtype: object


Codificamos la variable objetivo 

In [9]:
df['is_spam'] = df['is_spam'].astype(bool).map({True: 1, False: 0})

In [ ]:
# Buscamos duplicadas
df.duplicated().sum()

np.int64(630)

In [ ]:

df_clean = df.drop_duplicates();

df_clean.duplicated().sum()

np.int64(0)

PROCESADO DEL TEXTO 


In [17]:
def preprocess_text (url):

    url = url.lower();
      
    url = re.sub(r'https?://|www\.', '', url); # Borrado de Protocolo Web (http, https, www)
    url = re.sub(r'\s*\d+\s*', ' ', url);  # Borrado de números aislados
    url = re.sub(r'\b[a-zA-Z]*\d+[a-zA-Z]*\b', ' ', url);  # Borramos palabras que tengan letras y numeros mezclados
    url = re.sub(r'#.*', ' ', url); # Borramos signos
    url = re.sub(r'\b[a-zA-Z]\b', '', url); # Borramos letras aisladas

    # Segmentamos los URLs
    tokens = re.split(r'[\/\.\?\=\&\-_]', url);
    
    # Eliminamos las Stop Words
    stop_words = set(stopwords.words('english'));
    tokens = [token for token in tokens if token not in stop_words]; 

    # Lematización de palabras
    lemmatizer = WordNetLemmatizer(); 
    tokens = [lemmatizer.lemmatize(token) for token in tokens]; 

    # Eliminamos espacios y letras sueltas
    tokens = [token.strip() for token in tokens if token and len(token) > 1]; # Longuitudes de que sean mayor a 1

    return ' '.join(tokens);

In [28]:

url_random = df_clean.sample(1).url.values[0];

print(f'URL preprocesada:\n {url_random}');
print('-'*50);

url_random_prepro = preprocess_text(url_random);


print(f'URL post:\n {url_random_prepro}');

URL preprocesada:
 https://www.vox.com/culture/21294429/eurovision-review-interview-will-ferrell-rachel-mcadams-netflix
--------------------------------------------------
URL post:
 vox com culture eurovision review interview ferrell rachel mcadams netflix


In [29]:
# Creamos una nueva columna con el URL procesado
df_clean['url_postpro'] = df_clean['url'].progress_apply(preprocess_text);

100%|██████████| 2369/2369 [00:00<00:00, 8039.47it/s]
/tmp/ipykernel_5350/1906112549.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['url_postpro'] = df_clean['url'].progress_apply(preprocess_text);


In [30]:
# Borramos la columna url
df_clean = df_clean.drop(['url'], axis = 1);

In [31]:
df_clean

,is_spam,url_postpro
0,1,briefingday us list manage com unsubscribe
1,1,hvper com
2,1,briefingday com
3,0,briefingday com
4,1,briefingday com fan
...,...,...
2993,0,theverge com disney deepfake face swapping res...
2994,0,smartcitiesworld net news news deepfake techno...
2996,0,techcrunch com optimistic view deepfakes
2997,0,technologyreview com startup claim deepfakes p...


In [32]:
# Analizamos nuevamente los duplicados post procesamiento
df_clean.duplicated().sum()

np.int64(80)

In [33]:
df_postpro = df_clean.drop_duplicates();

In [ ]:
# Separacion Train Test
X_train, X_test, y_train, y_test = train_test_split(df_postpro ['url_postpro'], df_postpro ['is_spam'], test_size = 0.2, random_state = 42);

In [35]:

vectorizer = TfidfVectorizer();

X_train_vec = vectorizer.fit_transform(X_train);

X_test_vec = vectorizer.transform(X_test)

MODELO SIN OPTIMIZAR

In [36]:

svm_model = SVC();


svm_model.fit(X_train_vec, y_train);

y_pred = svm_model.predict(X_test_vec);


print("\n Métricas de la Predicción+:");
print(classification_report(y_test, y_pred));
print(f" Exactitud: {round(100*accuracy_score(y_test, y_pred), 2)} %");


 Métricas de la Predicción+:
              precision    recall  f1-score   support

           0       0.94      0.99      0.96       412
           1       0.86      0.39      0.54        46

    accuracy                           0.93       458
   macro avg       0.90      0.69      0.75       458
weighted avg       0.93      0.93      0.92       458

 Exactitud: 93.23 %


El modelo tiene una alta precisión general (93%), pero detecta mal los casos de SPAM. Aunque acierta en un 86% cuando dice que algo es SPAM, solo identifica correctamente el 39% de todos los SPAM reales, lo que indica un fuerte sesgo hacia los NO SPAM.

MODELO OPTIMIZADO

In [37]:

svm_model = SVC();

param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': ['scale', 'auto', 0.1, 1]
};

grid_search = GridSearchCV(estimator = svm_model, param_grid = param_grid, cv = 5, verbose = 1, n_jobs = -1);

grid_search.fit(X_train_vec, y_train);

print(f"\n\nMejores hiperparámetros: {grid_search.best_params_}\n\n");

best_grid_SVC= grid_search.best_estimator_;

y_pred_grid = best_grid_SVC.predict(X_test_vec);

print("\n Métricas de la Predicción - Con Optimización (GridSearch):");
print(classification_report(y_test, y_pred_grid));
print(f" Exactitud: {round(100*accuracy_score(y_test, y_pred_grid), 2)} %");

Fitting 5 folds for each of 64 candidates, totalling 320 fits


Mejores hiperparámetros: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}



 Métricas de la Predicción - Con Optimización (GridSearch):
              precision    recall  f1-score   support

           0       0.96      0.96      0.96       412
           1       0.62      0.63      0.62        46

    accuracy                           0.92       458
   macro avg       0.79      0.79      0.79       458
weighted avg       0.92      0.92      0.92       458

 Exactitud: 92.36 %


Tras aplicar GridSearch, la precisión general bajó ligeramente, pero el modelo mejoró mucho en detectar SPAM: el Recall subió del 39% al 63%. Aunque esto trajo más falsos positivos, permitió identificar más mensajes SPAM que antes pasaban desapercibidos.